# CNN Feature Map Visualization

This notebook provides deep learning feature visualization for understanding how VGG16 and ResNet50 process railway images.

## Analysis Goals:
- **Feature Map Visualization**: Display CNN layer activations from VGG16 and ResNet50
- **Layer Comparison**: Compare early, middle, and late layer responses
- **Model Comparison**: Understand differences between VGG16 and ResNet50 feature extraction
- **Single Image Focus**: Deep analysis of one image at a time

## Key Questions:
1. What visual features do different CNN layers detect in railway images?
2. How do VGG16 and ResNet50 differ in their feature extraction?
3. Which layers are most important for railway infrastructure recognition?

In [ ]:
# Import Required Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Reduce TensorFlow logging

# Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

# Set matplotlib style
plt.style.use('default')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Configuration
IMG_SIZE = (224, 224)  # Standard input size for VGG16 and ResNet50
RANDOM_SEED = 42

# ============================================================================
# MODIFY THIS PATH TO ANALYZE DIFFERENT IMAGES
# ============================================================================
EXAMPLE_IMAGE_PATH = "./datasets/clustering_sample_1000/ava_102_0000003090_C.png"

# Set random seeds
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"Target image size: {IMG_SIZE}")
print(f"Example image path: {EXAMPLE_IMAGE_PATH}")

In [ ]:
def load_image(image_path: str, target_size: tuple = IMG_SIZE):
    """
    Load and resize image for CNN analysis.
    
    Args:
        image_path: Path to the image file
        target_size: Target size for resizing (width, height)
    
    Returns:
        PIL Image: Resized image ready for CNN processing
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    # Load and resize image
    img_pil = Image.open(image_path)
    img_resized = img_pil.resize(target_size, Image.Resampling.LANCZOS)
    
    return img_resized

In [ ]:
class FeatureMapVisualizer:
    """Visualize feature maps from different CNN layers."""
    
    def __init__(self):
        self.models = {}
        self.layer_models = {}
        
    def load_models(self):
        """Load pre-trained VGG16 and ResNet50 models."""
        print("Loading pre-trained models...")
        
        # Load VGG16
        self.models['vgg16'] = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        
        # Load ResNet50
        self.models['resnet50'] = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        
        print("Models loaded successfully!")
        print(f"VGG16 layers: {len(self.models['vgg16'].layers)}")
        print(f"ResNet50 layers: {len(self.models['resnet50'].layers)}")
        
    def get_layer_outputs(self, model_name, layer_names):
        """
        Create models that output specific layer activations.
        
        Args:
            model_name: 'vgg16' or 'resnet50'
            layer_names: List of layer names to extract
        """
        base_model = self.models[model_name]
        
        # Get the outputs of specified layers
        layer_outputs = [base_model.get_layer(name).output for name in layer_names]
        
        # Create a model that returns these outputs
        activation_model = Model(inputs=base_model.input, outputs=layer_outputs)
        
        self.layer_models[f"{model_name}_layers"] = activation_model
        return activation_model
    
    def visualize_feature_maps(self, img, model_name, layer_names, max_filters=16):
        """
        Visualize feature maps from specific layers.
        
        Args:
            img: Input PIL image
            model_name: 'vgg16' or 'resnet50'
            layer_names: List of layer names to visualize
            max_filters: Maximum number of filters to show per layer
        """
        # Get preprocessing function
        preprocess_func = vgg16_preprocess if model_name == 'vgg16' else resnet50_preprocess
        
        # Prepare image for model
        img_array = image.img_to_array(img.resize((224, 224)))
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_func(img_array)
        
        # Get layer outputs
        activation_model = self.get_layer_outputs(model_name, layer_names)
        activations = activation_model.predict(img_array, verbose=0)
        
        # Visualize each layer
        for layer_idx, (layer_name, activation) in enumerate(zip(layer_names, activations)):
            self._plot_feature_maps(activation, layer_name, max_filters)
            
        return activations
    
    def _plot_feature_maps(self, activation, layer_name, max_filters=16):
        """Plot feature maps for a single layer."""
        # Get dimensions
        n_features = activation.shape[-1]
        size = activation.shape[1]
        
        # Limit number of filters to display
        n_cols = 8
        n_filters = min(max_filters, n_features)
        n_rows = (n_filters + n_cols - 1) // n_cols
        
        # Create the plot
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 2, n_rows * 2))
        if n_rows == 1:
            axes = axes.reshape(1, -1)
        
        for i in range(n_filters):
            row = i // n_cols
            col = i % n_cols
            
            # Display feature map
            feature_map = activation[0, :, :, i]
            axes[row, col].imshow(feature_map, cmap='viridis')
            axes[row, col].set_title(f'Filter {i}', fontsize=8)
            axes[row, col].axis('off')
        
        # Hide unused subplots
        for i in range(n_filters, n_rows * n_cols):
            row = i // n_cols
            col = i % n_cols
            axes[row, col].axis('off')
        
        plt.suptitle(f'{layer_name} - Feature Maps (Shape: {activation.shape})', fontsize=14)
        plt.tight_layout()
        plt.show()
    
    def compare_model_features(self, img):
        """
        Compare features extracted by VGG16 and ResNet50.
        
        Args:
            img: PIL Image
        """
        print("\n" + "="*60)
        print("COMPARING CNN MODEL FEATURES")
        print("="*60)
        
        # VGG16 key layers
        vgg16_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3']
        
        # ResNet50 key layers
        resnet50_layers = ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out', 'conv5_block3_out']
        
        print("\nVisualizing VGG16 Feature Maps...")
        vgg16_activations = self.visualize_feature_maps(img, 'vgg16', vgg16_layers)
        
        print("\nVisualizing ResNet50 Feature Maps...")
        resnet50_activations = self.visualize_feature_maps(img, 'resnet50', resnet50_layers)
        
        return vgg16_activations, resnet50_activations

In [ ]:
# Set image path to analyze
image_path = EXAMPLE_IMAGE_PATH

print(f"Analyzing image: {image_path}")
print(f"Image exists: {os.path.exists(image_path)}")

if not os.path.exists(image_path):
    print("\nImage not found! Please check the path or choose from available images:")
    # Try to list available images
    sample_dir = Path("./datasets/clustering_sample_1000")
    if sample_dir.exists():
        sample_images = list(sample_dir.glob("*_C.png"))[:10]  # Show first 10
        print("\nAvailable sample images:")
        for img in sample_images:
            print(f"  {img}")
        if len(sample_images) > 0:
            image_path = str(sample_images[0])
            print(f"\nUsing first available image: {image_path}")

In [ ]:
# Load image for CNN feature visualization
print("Loading image for CNN feature visualization...")

# Load image
img = load_image(image_path)

print(f"Image loaded and resized to: {img.size}")

# Display the original image
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.title(f"Input Image: {Path(image_path).name}")
plt.axis('off')
plt.show()

In [ ]:
# Initialize and load CNN models
print("\n" + "=" * 80)
print("INITIALIZING CNN MODELS")
print("=" * 80)

visualizer = FeatureMapVisualizer()
visualizer.load_models()

In [ ]:
# Visualize CNN feature maps
print("\n" + "=" * 80)
print("CNN FEATURE MAP VISUALIZATION")
print("=" * 80)

# Compare model features
vgg16_activations, resnet50_activations = visualizer.compare_model_features(img)

print("\n" + "=" * 80)
print("FEATURE ANALYSIS COMPLETE")
print("=" * 80)

print(f"\nAnalyzed image: {Path(image_path).name}")
print(f"VGG16 feature maps generated from {len(vgg16_activations)} layers")
print(f"ResNet50 feature maps generated from {len(resnet50_activations)} layers")

## Understanding CNN Feature Maps

### What Each Layer Detects:

**Early Layers (conv1, block1):**
- **Basic edges**: Horizontal, vertical, diagonal lines
- **Simple textures**: Basic patterns and gradients
- **Color transitions**: Boundaries between different regions
- **Railway context**: Rail edges, tie boundaries, ballast textures

**Middle Layers (conv3, block3):**
- **Complex patterns**: Repeated structures like railway ties
- **Shape recognition**: Geometric forms, curves, intersections
- **Texture combinations**: Complex surface patterns
- **Railway context**: Track geometry, infrastructure patterns

**Deep Layers (conv5, block5):**
- **High-level features**: Complete objects and structures
- **Semantic understanding**: Infrastructure types, conditions
- **Context recognition**: Urban vs rural, maintenance levels
- **Railway context**: Track types, environmental conditions

### Interpreting Filter Responses:

**Bright regions in feature maps** = Strong feature detection
**Dark regions in feature maps** = Weak/no feature detection
**Scattered bright spots** = Feature detected in multiple locations
**Uniform activation** = Feature present throughout the image

### Model Differences:

**VGG16:**
- Simpler architecture with clear hierarchical feature learning
- Good for understanding basic feature progression
- More interpretable layer-by-layer feature evolution

**ResNet50:**
- Skip connections allow for more complex feature combinations
- Better at capturing both low-level and high-level features
- More robust feature extraction due to residual learning

### Why This Matters for Clustering:

1. **Feature Quality**: Understanding what features are detected helps choose optimal layers
2. **Model Choice**: VGG16 vs ResNet50 may cluster images differently based on their feature extraction
3. **Layer Selection**: Earlier vs later layers capture different aspects of the image

### To Analyze Different Images:

Simply modify the `EXAMPLE_IMAGE_PATH` variable in the configuration cell and re-run the notebook.